In [ ]:
from src.Commands.NERInference import NERTester
from src.Commands.PaperProcessorPipeline import PaperProcessorPipeline
from concurrent.futures import ThreadPoolExecutor
from src.Commands.QuestionAnsweringModel import QAPaperExtractor
import datetime

from src.Commands.TaggingSystem import Tagging
def main():
    
    # Get today's date object
    today = datetime.date.today()
    # Format the date object into the desired string format
    formatted_date = today.strftime("%Y-%m-%d")

    pipeline = PaperProcessorPipeline(
        table_name='all_db',
        column_mapping={'id': 'primary_id'}
    )

    sources = [
        {
            "query": "SELECT primary_id, \"doi\", \"source\" FROM all_db WHERE \"doi\" IS NOT NULL AND \"doi\" != ''",
            "csv_file_path": "Data/output/papers_data_all",
            "db_name": "all_db_qa"
        }
    ]
    #  AND primary_id=1
    # Use parallel processing to process multiple sources simultaneously
    with ThreadPoolExecutor(max_workers=4) as executor:
        tagger = QAPaperExtractor()
        executor.map(
            lambda source: pipeline.process_source_in_batches(
                query=source["query"],
                csv_file_path=source["csv_file_path"],
                db_name=source["db_name"],
                tagger=tagger,
                batch_size=10
            ),
            sources
        )

if __name__ == "__main__":
    main()